# Vegetation type mapping from field and remote sensing data

**This Jupiter notebook is a WORK IN PROGRESS**
the project was initially done with R, I am currently translating it to python.

Based on my scientific article:

Shevtsova, I., Heim, B., Kruse, S., Schröder, J., Troeva, E., Pestryakova, L., Zakharov, E. and Herzschuh, U.: Strong shrub expansion in tundra-taiga, tree infilling in taiga and stable tundra in central Chukotka (north-eastern Siberia) between 2000 and 2017, Environmental Research Letters, 15(8), 085006, <a href="doi:10.1088/1748-9326/ab9059">doi:10.1088/1748-9326/ab9059</a>, 2020. (eng.)

## Vegetation data

Import needed packages

In [1]:
import pandas as pd
import numpy as np
#import PANGAEA special module to get the data from their repository
from pangaeapy import PanDataSet

**Retrieve the vegeation field data by number from PANGAEA repo and quick check**

The link to the dataset:
Shevtsova, Iuliia; Herzschuh, Ulrike; Heim, Birgit; Kruse, Stefan; Schröder, Julius; Troeva, Elena I; Pestryakova, Luidmila A; Zakharov, Evgenii S (2019): Foliage projective cover of 57 vegetation sites of central Chukotka from 2016. Alfred Wegener Institute - Research Unit Potsdam, PANGAEA, <a href="https://doi.org/10.1594/PANGAEA.908570">https://doi.org/10.1594/PANGAEA.908570</a>


In [2]:
veg_data=PanDataSet(908570)

#print a title to check if this is right dataset
print(veg_data.title)
print()
print(veg_data.data.head())

Foliage projective cover of 57 vegetation sites of central Chukotka from 2016

          Event Site  Latitude  Longitude      Code Plot size  Plot rad  \
0  16-KP-01-V01  V01   67.3618   168.2542  16-KP-01       NaN      15.0   
1  16-KP-01-V02  V02   67.3660   168.2366  16-KP-01       NaN      15.0   
2  16-KP-01-V03  V03   67.3664   168.2948  16-KP-01     24x26       NaN   
3  16-KP-01-V04  V04   67.3736   168.3100  16-KP-01       NaN      15.0   
4  16-KP-01-V05  V05   67.3769   168.3122  16-KP-01       NaN      15.0   

   Elevation  Slope Aspect  ...  Orobanchaceae cov  Orthilia cov  \
0      492.0   2.50      S  ...                0.4           0.0   
1      559.0   1.67      S  ...                0.3           0.2   
2      485.0   1.43     SW  ...                0.1           0.1   
3      509.0   2.57    SSW  ...                0.0           0.4   
4      584.0   6.89    SSO  ...                0.0           0.4   

   Polygonaceae cov  Primulaceae cov  Pyrola spp. cov  Ranunc

Get column names

In [3]:
print(veg_data.data.columns)

Index(['Event', 'Site', 'Latitude', 'Longitude', 'Code', 'Plot size',
       'Plot rad', 'Elevation', 'Slope', 'Aspect', 'A. fruticosa cov',
       'P. pumila cov', 'L. cajanderii cov', 'Salix cov', 'Salix cov_2',
       'B. exilis cov', 'C. tetragona cov', 'E. nigrum cov', 'L. palustre cov',
       'V. uliginosum cov', 'V. vitis-idaea', 'R. adamsii cov',
       'R. arctica cov', 'A. alpina cov', 'Lichen cov', 'Moss cov',
       'Poaceae+Cyperaceae cov', 'Asteraceae cov', 'Brassicaceae cov',
       'Campanulaceae cov', 'Caprifoliaceae cov',
       'Caryophyllaceae+Montiaceae cov', 'Equisetaceae cov', 'Fabaceae cov',
       'Gentianaceae cov', 'Grossulariaceae cov', 'Orobanchaceae cov',
       'Orthilia cov', 'Polygonaceae cov', 'Primulaceae cov',
       'Pyrola spp. cov', 'Ranunculaceae cov', 'Rosaceae cov',
       'D. octopetala cov', 'Rubiaceae cov', 'Saxifragaceae cov'],
      dtype='object')


In [4]:
veg_data = veg_data.data

In [5]:
#filtering out unnessasary data
cols= [i for i in range(1,5)]+[i for i in range(7,46)]
rows=[i for i in range(0,57)]
print(cols)#preserve these
veg_data = veg_data.iloc[rows,cols]
print(veg_data)

[1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]
   Site  Latitude  Longitude      Code  Elevation  Slope Aspect  \
0   V01   67.3618   168.2542  16-KP-01      492.0   2.50      S   
1   V02   67.3660   168.2366  16-KP-01      559.0   1.67      S   
2   V03   67.3664   168.2948  16-KP-01      485.0   1.43     SW   
3   V04   67.3736   168.3100  16-KP-01      509.0   2.57    SSW   
4   V05   67.3769   168.3122  16-KP-01      584.0   6.89    SSO   
5   V06   67.3500   168.1885  16-KP-01      593.0   3.00      N   
6   V07   67.3456   168.1842  16-KP-01      692.0   2.22    WSW   
7   V08   67.3449   168.1802  16-KP-01      672.0   1.67     SW   
8   V09   67.3538   168.2157  16-KP-01      488.0   0.67     SO   
9   V10   67.3452   168.2013  16-KP-01      547.0   5.14    SSO   
10  V11   67.3500   168.2009  16-KP-01      600.0   1.00    NOO   
11  V12   67.3531   168.2264  

**Hellinger transformation**

The square root of observed values that have been devided by row(site) sums.
IN R: "decostand" function (vegan).(Legendre & Gallagher, 2001)
Recommendended for clustering or ordination of species abundance.
    
Purpose: removes distances in total abundance, bur maintains differences in relative species composition among sites.

In [6]:
#take only cover data from the data frame
cov=[i for i in range(7,43)]
print(cov)
veg_data_cov = veg_data.iloc[:,cov]
print(veg_data_cov.columns)

[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
Index(['A. fruticosa cov', 'P. pumila cov', 'L. cajanderii cov', 'Salix cov',
       'Salix cov_2', 'B. exilis cov', 'C. tetragona cov', 'E. nigrum cov',
       'L. palustre cov', 'V. uliginosum cov', 'V. vitis-idaea',
       'R. adamsii cov', 'R. arctica cov', 'A. alpina cov', 'Lichen cov',
       'Moss cov', 'Poaceae+Cyperaceae cov', 'Asteraceae cov',
       'Brassicaceae cov', 'Campanulaceae cov', 'Caprifoliaceae cov',
       'Caryophyllaceae+Montiaceae cov', 'Equisetaceae cov', 'Fabaceae cov',
       'Gentianaceae cov', 'Grossulariaceae cov', 'Orobanchaceae cov',
       'Orthilia cov', 'Polygonaceae cov', 'Primulaceae cov',
       'Pyrola spp. cov', 'Ranunculaceae cov', 'Rosaceae cov',
       'D. octopetala cov', 'Rubiaceae cov', 'Saxifragaceae cov'],
      dtype='object')


In [7]:
#convert to Numpy array
veg_array = veg_data_cov.to_numpy() 
#print(veg_array)

#Devide all values in a row by row sum
row_by_rowsum = veg_array/veg_array.sum(axis=1, keepdims=True)
#Square root of the values
veg_hel_transf = np.sqrt(row_by_rowsum)
print(veg_hel_transf)

[[0.12803688 0.         0.4267896  ... 0.         0.         0.        ]
 [0.1014301  0.07172191 0.20286021 ... 0.         0.03207501 0.04536092]
 [0.         0.07224408 0.28897631 ... 0.         0.14448815 0.        ]
 ...
 [0.         0.         0.         ... 0.61655466 0.         0.06228143]
 [0.         0.         0.         ... 0.42464272 0.         0.        ]
 [0.         0.         0.         ... 0.19094393 0.         0.20429544]]


**Detrended correspondence Analysis**

Needed to know if the field data distributed on a short linear gradient or is it more a unimodal distribution.
In R: function "decorana".

In [8]:
#--currently not working due to ModuleNotFoundError--#

#import cornpy as cp
#site_scores, species_scores = cp.decorana(veg_hel_transf)

ModuleNotFoundError: No module named 'cornpy'